## train_classifier_ROC_trainSet

15/06/16 by greghor


on commence par regarder les documents ou l on observe le mot center
utilise les fichiers .html
integre corrections typographiques pour coller à la solution
integre ROC à partir des currencies

In [1]:
# SYSTEM
from os import listdir
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

# data from Albert
from code.dataExtract import data
import code.dataExtract as de
from code import Text_reader as tr
# Read the data
d = data(folder='../../train/') # training set with all the labels
dataPath='../../train/'



The PythonMagick package is missing, get_text_adv function can not be used 
The PyPDF2 package is missing, get_text_adv function can not be used 
The html2text package is missing, get_text_adv and get_text functions can not be used.
Please consider the instalation of this package !


In [2]:
# fonctions used in the notebook

def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context+=s[:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context+=s[loc-width:]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context+=s[loc-width:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)


    
def useSolutionTypo(s):
    s=s.lower()
    if s=="sao paulo":
        s="sao-paulo"
    elif s=='target 1' or s =='target 2' or s == 'target' or s=='target1':
        s='target holidays'
    return s



def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        clean_text = clean_text.lower()
        clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text

def removeUnwantedCity(city, unwantedCity):
    #
    assert isinstance(city,list) or isinstance(city,np.ndarray), "city should be an np.array or a list"
    assert isinstance(unwantedCity,list),  "unwanted city should be a list"
        
    for gi in range(len(city)):
        for unwanted in unwantedCity:
            if '|' + unwanted in city[gi]:
                city[gi]=city[gi].replace('|'+ unwanted,'')
            elif unwanted + '|' in city[gi]:
                city[gi]=city[gi].replace(unwanted + '|','')
            elif unwanted in city[gi]:
                city[gi]=city[gi].replace(unwanted,'')
    return city
                

In [3]:
# get the solution
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [4]:
# link roc and operationnal curency
opCurrency=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
opCurr_df=pd.DataFrame(opCurrency.items(),columns=['ISIN','OpCurrency'])
link_roc_opCurr=pd.read_csv('currency_ROC_fullList.csv',delimiter=";")
# merge with labels on operationnal currency
roc_fromCurrency=pd.merge(left=link_roc_opCurr,right=opCurr_df,left_on='Currency Code', right_on='OpCurrency')
# keep only currency code and default ROC columns
col=["ISIN","OpCurrency","Default ROC",]
roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.rename(columns={'Default ROC':'currency_ROC'},inplace=True)
len(roc_fromCurrency)

426

In [5]:
gf=0


context_pat=dict()
text=dict()
htmlFiles=tr.get_files(dataPath + '/html')



for isin in d.docid.keys()[:]:
        
    text[isin]=document_from_hmtl(htmlFiles[d.docid[isin][0]])
    context_pat[isin]=''

    pattern=["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']
    
    ## recupere le contexte si patterns, prend l ensemble du text sinon
    
    context_pat[isin]+=text[isin]
    
    """  
    for pat in pattern:
        context_pat[isin]+=getContext(text[isin],pat.lower(),'',150)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
    if len(context_pat[isin])==0:
        context_pat[isin]+=text[isin]"""
            
    gf+=1
    progression=float(gf)/len(d.docid.keys())*100
    if gf%100==0:
        print float(gf)/len(d.docid.keys())
       


0.0183284457478
0.0366568914956
0.0549853372434
0.0733137829912
0.091642228739
0.109970674487
0.128299120235
0.146627565982
0.16495601173
0.183284457478
0.201612903226
0.219941348974
0.238269794721
0.256598240469
0.274926686217
0.293255131965
0.311583577713
0.32991202346
0.348240469208
0.366568914956
0.384897360704
0.403225806452
0.421554252199
0.439882697947
0.458211143695
0.476539589443
0.494868035191
0.513196480938
0.531524926686
0.549853372434
0.568181818182
0.58651026393
0.604838709677
0.623167155425
0.641495601173
0.659824046921
0.678152492669
0.696480938416
0.714809384164
0.733137829912
0.75146627566
0.769794721408
0.788123167155
0.806451612903
0.824780058651
0.843108504399
0.861436950147
0.879765395894
0.898093841642
0.91642228739
0.934750733138
0.953079178886
0.971407624633
0.989736070381


In [6]:
len(PyFind(context_pat.values(),lambda x : len(x)==0))

0

#### prepare data for classifier


In [7]:
# get the labels and merge with context
citiesGT=pd.read_csv('citiesGreg.csv')
context_df=pd.DataFrame(context_pat.items(),columns=['isin','context'])
training_df=pd.merge(context_df,citiesGT,on='isin')
training_df=training_df[['isin','context','citiesName']]
print training_df.head()

           isin                                            context  \
0  XS1313800895   no prospectus is required in accordance with ...   
1  XS1267246327   pricing supplement dated december 10 2015 exe...   
2  XS1267238654   pricing supplement dated december 9 2015 gold...   
3  XS1293107071   final terms dated 16 december 2015 bnp pariba...   
4  XS1326548325   no prospectus is required in accordance with ...   

                 citiesName  
0              LONDON|TOKYO  
1  NEW YORK|TARGET HOLIDAYS  
2                  NEW YORK  
3           TARGET HOLIDAYS  
4                 HONG KONG  


In [8]:
# create the list and feed the trainer

context=training_df.as_matrix(columns=['context']).ravel()
# lowerise, and replace pipe by space for the labels
roc=[roc.lower().replace('|',' ') for roc in training_df['citiesName'].values]
roc=np.asarray(roc)


In [9]:
roc

array(['london tokyo', 'new york target holidays', 'new york', ...,
       'new york', 'hong kong', 'beijing hong kong'], 
      dtype='|S54')

In [10]:
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(context, roc,\
                                                                     test_size=0.3, random_state=53)

In [11]:
# scikit impory
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from time import time


In [12]:
#### trainac

In [13]:
# train 
text_clf = Pipeline([('vect', CountVectorizer()), \
                     ('tfidf', TfidfTransformer()), \
                     ('clf', SGDClassifier(average=True))])

text_clf = text_clf.set_params(clf__alpha = 1e-04, \
                               clf__n_iter = 100, \
                               clf__penalty = 'l2', \
                               clf__n_jobs=-1,\
                               tfidf__norm = 'l2', \
                               tfidf__use_idf = False, \
                               vect__max_df = 0.4, \
                               vect__ngram_range = (1, 2))

text_clf = text_clf.fit(X_train.ravel(), y_train.ravel())

y_train_pred = text_clf.predict(X_train)

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

print(np.mean(y_train == y_train_pred))

print(classification_report(y_train, y_train_pred))

0.986180904523
             precision    recall  f1-score   support

auckland london new york wellington       1.00      1.00      1.00         1
beijing hong kong       1.00      1.00      1.00         3
beijing hong kong london new york toronto       1.00      1.00      1.00         1
brasilia london new york tokyo       1.00      1.00      1.00         3
   budapest       1.00      1.00      1.00         2
caire new york       1.00      1.00      1.00         1
  hong kong       0.95      1.00      0.97       136
hong kong london new york       1.00      1.00      1.00         2
hong kong london new york toronto       1.00      1.00      1.00         1
hong kong london target holidays       1.00      1.00      1.00         1
hong kong london tokyo       1.00      1.00      1.00         1
hong kong new york       1.00      0.93      0.96        14
hong kong sydney       1.00      1.00      1.00         1
istanbul london new york target holidays tokyo       1.00      1.00      1.00   

In [15]:
y_test_pred = text_clf.predict(X_test)
print(text_clf.score(X_test, y_test))

0.804093567251


In [16]:
joblib.dump(text_clf, 'sgd_model/roc_70Trainpipeline.pkl') 

['sgd_model/roc_70Trainpipeline.pkl',
 'sgd_model/roc_70Trainpipeline.pkl_01.npy',
 'sgd_model/roc_70Trainpipeline.pkl_02.npy',
 'sgd_model/roc_70Trainpipeline.pkl_03.npy',
 'sgd_model/roc_70Trainpipeline.pkl_04.npy',
 'sgd_model/roc_70Trainpipeline.pkl_05.npy',
 'sgd_model/roc_70Trainpipeline.pkl_06.npy',
 'sgd_model/roc_70Trainpipeline.pkl_07.npy',
 'sgd_model/roc_70Trainpipeline.pkl_08.npy']